# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> IMT2118 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2024-1**<br>
**Profesora:** Paula Aguirre <br>
**Ayudante:** Diego Pizarro <br>
**Integrantes:** Alu Palu,Bastian Baez Oses
<br>





---

In [ ]:
import geopandas as gpd
import geemap
import ee
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import imageio
import pandas as pd
import folium

In [ ]:
#Inicializamos GEE
ee.Authenticate();
ee.Initialize(project='ee-bastianbaezos') #use su código de proyecto

In [ ]:
def getAOI(dat,crs=4326):
    aoi_coords = list(dat.geometry.unary_union.exterior.coords)
    aoi = ee.Geometry.Polygon(aoi_coords)
    return aoi

# La siguiente función fue sacada de:
# https://github.com/gee-community/geemap/issues/1095#issuecomment-1151812523
def fmask(image):
    # see https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True).updateMask(qaMask)

def get_images(year_init, year_end, aoi):

    # Funcion creada usando de base la de la ayudantia 4
    years = list(range(year_init, year_end + 1))
    annual_imgs = {year: '' for year in years}

    for year in years:
        startDate = f'{year}-01-01'
        endDate = f'{year}-12-31'

        sentinelImageCollection = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(aoi).filterDate(startDate, endDate)

        # Aplicamos fmask
        sentinelImageCollection = sentinelImageCollection.map(fmask)

        bandas = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']

        # Seleccionamos las bandas de interés
        sentinelImageCollection = sentinelImageCollection.select(bandas)

        # Geomedian ee.Reducer.geometricMedian()
        image = sentinelImageCollection.reduce(ee.Reducer.geometricMedian(numX=6)).clip(aoi)

        # Cambiar nombre de las bandas a "bandas"
        image = image.rename(bandas)

        # Agregar imagen al diccionario
        annual_imgs[year] = image

    return annual_imgs